# Capstone Project

This notebook will be used for my IBM Data Science Capstone Project!

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course")

Hello Capstone Project Course
